In [1]:
import polars as pl

In [2]:
data = pl.read_parquet("../tests/test_data/test_data.parquet")
data.head()

date,open,high,low,close,volume,split_ratio,dividend,symbol,log_returns,window_index,window_mean,detrended_log_returns,cum_sum,cum_sum_min,cum_sum_max,cum_sum_range,cum_sum_std,realized_volatility,vol_bucket
date,f64,f64,f64,f64,f64,f64,f64,str,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,str
2000-01-05,0.784238,0.835733,0.778569,0.786128,7.783216e8,0.0,0.0,"""AAPL""",0.014527,287,-0.00537,0.019896,-0.062811,-0.212293,0.08333,0.295623,0.091599,21.971445,"""low"""
2000-01-06,0.802191,0.808805,0.718098,0.718098,7.679728e8,0.0,0.0,"""AAPL""",-0.090514,287,-0.00537,-0.085144,-0.147955,-0.212293,0.08333,0.295623,0.091599,26.168462,"""mid"""
2000-01-07,0.729436,0.763452,0.721878,0.752113,4.607344e8,0.0,0.0,"""AAPL""",0.046281,287,-0.00537,0.051651,-0.096305,-0.212293,0.08333,0.295623,0.091599,28.475358,"""mid"""
2000-01-10,0.77101,0.7729,0.716208,0.738885,5.05064e8,0.0,0.0,"""AAPL""",-0.017744,287,-0.00537,-0.012375,-0.108679,-0.212293,0.08333,0.295623,0.091599,32.249623,"""mid"""
2000-01-11,0.725184,0.751168,0.684083,0.70109,4.415488e8,0.0,0.0,"""AAPL""",-0.052506,287,-0.00537,-0.047137,-0.155816,-0.212293,0.08333,0.295623,0.091599,37.15822,"""mid"""


In [3]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)

mandelbrot = calc_mandelbrot_channel(
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()

In [10]:
mandelbrot

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2023-12-29,"""AAPL""",191.0641,192.284637,214.0328
2023-12-29,"""AMD""",131.4548,147.410004,152.316
2023-12-29,"""AMZN""",146.3461,151.940002,159.3488
2023-12-29,"""GOOGL""",135.1156,139.529861,141.2999
2023-12-29,"""PCT""",2.4087,4.05,5.1265


In [4]:
import pickle

with open("../tests/test_data/mandelbrot_current.pkl", "r+b") as file:
    test_data = pickle.load(file)



In [5]:
test_data

HumblObject

id: 0667182b-9b7e-7e4e-8000-a61c859bee7f
results: {"IR":{"version":506,"dsl":{"Select":{"expr":[{"Column":"date"},{"Column":...
equity_data: {"IR":{"version":0,"dsl":{"MapFunction":{"input":{"DataFrameScan":{"df...
provider: yfinance
warnings: None
chart: None
extra: {}
context_params: {'symbol': ['AMD', 'AAPL', 'NVDA', 'SMCI', 'GME'], 'interval': '1d'...
command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...

## Creating Fixture on the Fly 

In [11]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL", "NVDA", "SMCI", "GME"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2021-02-20",
)
toolbox
result = toolbox.technical.mandelbrot_channel(historical=False, window="1m")
result

HumblObject

id: 0667196b-79ea-75c5-8000-053e08ac3f33
results: {"IR":{"version":1245,"dsl":{"Select":{"expr":[{"Column":"date"},{"Column"...
equity_data: {"IR":{"version":739,"dsl":{"MapFunction":{"input":{"DataFrameScan":{"...
provider: yfinance
warnings: None
chart: None
extra: {}
context_params: {'symbol': ['AMD', 'AAPL', 'NVDA', 'SMCI', 'GME'], 'interval': '1d'...
command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...

In [12]:
result.to_polars()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2021-02-19,"""AAPL""",127.5741,127.643982,144.8156
2021-02-19,"""AMD""",84.379,89.580002,96.2867
2021-02-19,"""GME""",10.1433,10.1475,11.3692
2021-02-19,"""NVDA""",14.239,14.891013,15.5453
2021-02-19,"""SMCI""",30.4431,31.68,34.0473
